# DataFrame 만들기 

1. 재생목록에 대한 dataframe

2. track에 대한 dataframe : hierarchical indexing이용



In [5]:
from pandas import DataFrame
import pandas as pd
import json
import csv


import numpy as np
import re

# 1 재생목록 dataframe json으로 만듬

이모티콘 같은 게 csv로 파일 바꿀 때 아스키코드 때문에 문제가 생겨서

json 파일로 만듬

file_num에 해당하는 json file을 읽어서 dataframe을 반환


In [6]:
common_front = '../data/mpd.slice.'
common_back = '.json'


def get_playlist_file(file_num):   
    front = file_num*1000
    back = front + 999
    file_path = common_front + str(front) + '-' +str(back) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    playlists = data["playlists"]
    df_playlist = pd.DataFrame(playlists)
    return df_playlist

In [7]:
get_playlist_file(1)

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,NaN,31428282,1457827200,disney,16,65,4,1,189,1000,"[{u'pos': 0, u'artist_name': u'Original Broadw..."
1,false,NaN,38241566,1417824000,Indie Electro,18,8,2,2,165,1001,"[{u'pos': 0, u'artist_name': u'The Octopus Pro..."
2,false,NaN,3549358,1465430400,jack & jack,14,3,3,1,17,1002,"[{u'pos': 0, u'artist_name': u'Jack & Jack', u..."
3,false,NaN,51242585,1498435200,vibes,195,157,91,2,225,1003,"[{u'pos': 0, u'artist_name': u'LANY', u'track_..."
4,false,NaN,42601098,1498608000,Indie,118,92,74,1,165,1004,"[{u'pos': 0, u'artist_name': u'Youth Lagoon', ..."
5,false,NaN,5870710,1504310400,college,26,25,2,1,28,1005,"[{u'pos': 0, u'artist_name': u'Demi Lovato', u..."
6,false,NaN,19652753,1471996800,Summer 15,74,67,40,2,80,1006,"[{u'pos': 0, u'artist_name': u'Slaptop', u'tra..."
7,false,NaN,13158862,1509148800,New music,48,38,20,1,58,1007,"[{u'pos': 0, u'artist_name': u'Post Malone', u..."
8,false,NaN,18432208,1505692800,Now,82,79,36,3,84,1008,"[{u'pos': 0, u'artist_name': u'Cyrus', u'track..."
9,false,NaN,9413467,1507680000,Jams,36,31,25,1,41,1009,"[{u'pos': 0, u'artist_name': u'Justin Timberla..."


한 파일 내의 재생목록 수 1000개 인지 확인

column어떤거 있는 지 확인

In [8]:
print(get_playlist_file(1).index)
print(get_playlist_file(1).columns)

RangeIndex(start=0, stop=1000, step=1)
Index([u'collaborative', u'description', u'duration_ms', u'modified_at',
       u'name', u'num_albums', u'num_artists', u'num_edits', u'num_followers',
       u'num_tracks', u'pid', u'tracks'],
      dtype='object')


In [9]:
get_playlist_file(1)['num_tracks'].sum()#test

66622

# 2 total track dataframe 


In [10]:
import glob
all_file= glob.glob("../data/*")
len(all_file)

1001

hierarchical indexing에 사용할 index array만들기

모든 각각 tracks에 대해서 몇 번 playlist인지 indexing 하기 위함

In [11]:
def make_index_of_tracks(file_num):
    num_tracks = get_playlist_file(file_num)["num_tracks"]
    num_tracks = np.array(num_tracks)
    playlist_index_of_track = []

    for (playlist_index) in range(1000):
        for(track_index) in range(num_tracks[playlist_index]):
            playlist_index_of_track.append(playlist_index + file_num*1000)
        
    return np.array(playlist_index_of_track)

In [12]:
#print(make_index_of_tracks(0))
make_index_of_tracks(3)  #list 

array([3000, 3000, 3000, ..., 3999, 3999, 3999])

get_tracks_of_playlist(file_num)

: file_num에 해당하는 파일의 tracks column 가져오기


get_tracks_of_first_playlist(file_num)

: file_num에서 가장 첫번 째에 위치한 playlist를 dataframe으로 반환

In [13]:
def get_tracks_of_playlist(file_num):    #df_playlist0
    return get_playlist_file(file_num)['tracks']

def get_tracks_of_first_playlist(file_num):    #tmp
    tmp = DataFrame(get_playlist_file(file_num)['tracks'][0])
    return tmp

첫번째 재생목록의 track를 시작으로 

다음 재생목록의 tracks를 concatenate 

In [193]:
def concat_tracks(file_num):
    df_tracks = DataFrame(get_tracks_of_first_playlist(file_num))
    for i in range(1,1000):
        df_tmp = DataFrame(get_tracks_of_playlist(file_num)[i])
        df_tracks = pd.concat([df_tracks, df_tmp])
    df_tracks = df_tracks.reset_index(drop=True)
    return df_tracks

In [199]:
concat_tracks(1)

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri
0,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,154506,0,Fathoms Below - Broadway Cast Recording,spotify:track:5IbCV9Icebx8rR6wAp5hhP
1,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,79066,1,Daughters Of Triton - Broadway Cast Recording,spotify:track:6rKVAvjHcxAzZ1BHtwh5yC
2,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,94600,2,The World Above - Broadway Cast Recording,spotify:track:6Jlkb1Wh08RYHstWScsTvg
3,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,151480,3,Human Stuff - Broadway Cast Recording,spotify:track:0XhC8bfStML9ygBmfOt1JJ
4,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,297920,4,I Want the Good Times Back - Broadway Cast Rec...,spotify:track:0ABxAcsRWlqckkyONsfP67
5,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Sierra Boggess as Ariel,spotify:artist:1j4a3bBWYNjecFqxcjRtr5,204573,5,Part Of Your World,spotify:track:1iVkEiZS6ijZ76KeMsrXG6
6,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Alan Menken,spotify:artist:5sy77gt4bfsLcSQ8GIe4ZZ,120133,6,Storm at Sea - Broadway Cast Recording,spotify:track:0yLjkpa51CN1uiryMDOrtt
7,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,122813,7,Part Of Your World (Reprise) - Broadway Cast R...,spotify:track:2jxJF5RaHcqcUyg3T3FHpW
8,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,219640,8,She's in Love - Broadway Cast Recording,spotify:track:5eHx8TOZ7UIQLtd6aUWKkK
9,The Little Mermaid: Original Broadway Cast Rec...,spotify:album:3ULJeOMgroG27dpn27MDfS,Original Broadway Cast - The Little Mermaid,spotify:artist:3TymzPhJTMyupk7P5xkahM,195946,9,Her Voice - Broadway Cast Recording,spotify:track:0hfFDkS845UKM3BIKXmFBo


# 3. total tracks with playlist index 

In [200]:
col = get_tracks_of_first_playlist(0).columns

In [201]:
col

Index([u'album_name', u'album_uri', u'artist_name', u'artist_uri',
       u'duration_ms', u'pos', u'track_name', u'track_uri'],
      dtype='object')

In [202]:
make_index_of_tracks(1).size

66622

In [203]:
np.array(concat_tracks(1).index).size

66622

In [204]:
get_playlist_file(1)['num_tracks'].sum()

66622

track 정보가 담긴 dataframe. playlist_idx 열 추가.

In [205]:
def tracks_with_playlist_idx(file_num):
    frame = DataFrame(concat_tracks(file_num))
    frame['playlist_idx'] = make_index_of_tracks(file_num)
    return frame

In [206]:
playlist0 = tracks_with_playlist_idx(0)

In [207]:
playlist0

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_idx
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0


# 4. Only track_name in list

matrix factorization을 하기위한 기본틀 만들기

row: playlist

column: playlist 1000개(json 파일 1개)에 들어있는 모든 track_name

unique한 값만 뽑기(중복 제거) -> column에 넣기위한 track_name list 만들기

In [208]:
def get_unique_things(file_num, column):
    total_tracks = concat_tracks(file_num)  # 모든 트랙을 합쳐놓음.
    unique_things_of_total_tracks = total_tracks[column]  # 뽑고 싶은 column에 있는 값을 모아둔 리스트
    unique_things_of_total_tracks = list(set(unique_things_of_total_tracks))   # 중복제거
    
    return unique_things_of_total_tracks

In [209]:
track_name_of_total_tracks = get_unique_things(0,'track_name')

In [210]:
track_name_of_total_tracks

[u"My Man's Gone Now",
 u'Violet Hill',
 u'1000 Nights',
 u'Cissy Strut',
 u'Always Summer',
 u'Where It All Goes',
 u'There For You',
 u'Conditions of My Parole',
 u'Warrior (Battle Cry Remix)',
 u'Dangerously In Love',
 u'Bottoms Up - feat. Nicki Minaj',
 u'Mujer Divina',
 u'Backdrifts',
 u'Sunshine Girl',
 u'A Zip & A Double Cup (Remix) (feat. Tity Boi)',
 u"Lugia's Song From Pokemon 2000 on Piano",
 u"It's Gotta Be You",
 u'Toccata and Fugue in D Minor',
 u'Coming Over - filous Remix',
 u'Songs Of Zion',
 u'Still Think About You',
 u'Part 2: Fired Up... Feels Good',
 u'I Want It All (Remix)',
 u'Blow Us All Away',
 u'Statistica (Statistic)',
 u'Time Chamber',
 u'Fine Flow',
 u'Goshen',
 u'Me And My Broken Heart',
 u'If I ever feel better',
 u'Waving Through A Window',
 u'Big Booty Bitches - Spic N Span Original Eargasm',
 u'Zat You, Santa Claus? (feat. Ren\xe9 Marie)',
 u"I Feel Like Dancin'",
 u'do re mi',
 u'8TEEN',
 u'Do You Love Me Now?',
 u'Fantine\u2019s Death',
 u'Toxic Toas

* artist URI 개수와 unique artist name수 비교

In [211]:
artist_name_of_total_tracks = get_unique_things(0,'artist_name')

In [212]:
len(artist_name_of_total_tracks)

9722

In [213]:
artist_uri_of_total_tracks = get_unique_things(0,'artist_uri')

In [214]:
len(artist_uri_of_total_tracks)

9754

같은 artist여도 album이 다르면 uri가 다를 수 있음.

# 5. Matrix Factorization - initialized

playlist에 track이 있으면 1, 없으면 0

In [215]:
playlist0['exist'] = 1;  #playlist에 해당 트랙 존재 여부

In [216]:
playlist0

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_idx,exist
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0,1
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0,1
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0,1
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0,1
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0,1
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0,1
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0,1
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0,1
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0,1
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0,1


In [217]:
original_df = playlist0.pivot_table(index='playlist_idx', columns='track_name', values = 'exist').fillna(0)

In [218]:
original_df

track_name,"""40"" - Remastered 2008","""45""","""A Lovely Night""","""A Lovely Night"" (Reprise)","""Can't Repeat The Past?""","""Do I Love You Because You're Beautiful""","""Esh 'Dani, Alash Mshit""","""Gatsby Believed In The Green Light""","""He Was Tall""","""Heroes"" - Single Version, 2017 Remastered Version",...,사실은 The Truth Is,양화대교 (Yanghwa Brdg),오늘 헤어졌어요,왜 (Keep Your Head Down) [Rearranged],왜 이러니,우주보다 좋아해 (I Love You More Than Universe),으르렁 Growl,전야 前夜 The Eve,주문 Mirotic,"헤어지지 못하는 여자, 떠나가지 못하는 남자 Can't Breakup Girl, Can't Breakaway Boy (feat. 정인 Jung In)"
playlist_idx,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [219]:
original_matrix = np.array(original_df) # dataframe형식을 배열로(matrix)

In [882]:
original_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

각 track이 playlist에 담긴 횟수

In [879]:
col_sum = original_df.sum(axis=0) # 각 column의 합 구하기
col_sum.max()

71.0

In [883]:
for(track) in range(len(col_sum)):
    if(col_sum[track] > 30):
        print col_sum.index[track]
        print col_sum[track]

Bad and Boujee (feat. Lil Uzi Vert)
36.0
Black Beatles
31.0
Bounce Back
38.0
Broccoli (feat. Lil Yachty)
49.0
Caroline
35.0
Cheap Thrills
32.0
Closer
71.0
Congratulations
46.0
Don't Let Me Down
43.0
Down
35.0
Drive
32.0
Forever
41.0
Get Low - Street
31.0
Gold
38.0
Gold Digger
35.0
HUMBLE.
52.0
Home
43.0
Jumpman
38.0
Let Me Love You
37.0
Location
34.0
Mask Off
38.0
Mercy
32.0
Never Be Like You
34.0
Ni**as In Paris
31.0
No Problem (feat. Lil Wayne & 2 Chainz)
31.0
No Role Modelz
34.0
One Dance
50.0
Panda
36.0
Ride
47.0
Roses
48.0
Selfish
31.0
September
31.0
Skinny Love
37.0
Sorry
40.0
Starboy
32.0
Stronger
34.0
T-Shirt
33.0
Trap Queen
32.0
Waves
33.0
White Iverson
38.0
Work
31.0
Work from Home
32.0
XO TOUR Llif3
36.0
Yeah!
34.0
goosebumps
35.0
iSpy (feat. Lil Yachty)
39.0


# 6. SVD

In [243]:
U, s, Vt = np.linalg.svd(original_matrix, full_matrices=True)

In [244]:
U

array([[  2.30746935e-02,  -2.82896686e-02,   3.41363086e-02, ...,
         -1.10225241e-02,   4.27684411e-03,  -2.48043563e-03],
       [  4.57622785e-03,  -1.75348873e-02,   6.78605070e-03, ...,
         -1.25808424e-03,  -1.09906931e-03,  -7.64371695e-04],
       [  5.33197227e-04,  -1.01789813e-03,   4.69364154e-04, ...,
          3.01069014e-04,   2.25761075e-04,   7.13860487e-05],
       ..., 
       [  3.90461180e-02,  -9.73983639e-03,   2.85535739e-02, ...,
          9.37051134e-04,   1.98844352e-04,   2.42376737e-04],
       [  1.40760936e-02,  -8.13962827e-02,  -1.08252778e-01, ...,
         -2.27400021e-04,   1.20313181e-03,   3.52398027e-04],
       [  3.58833910e-03,  -3.02805793e-03,   1.93861417e-03, ...,
          2.80130552e-04,  -1.05846061e-03,  -6.28263808e-04]])

In [245]:
Vt

array([[  1.41790735e-06,   2.02969854e-04,   1.39784303e-04, ...,
          1.57075581e-05,   3.06593601e-05,   3.06593601e-05],
       [ -8.86476851e-06,  -3.22882724e-04,  -1.96322261e-04, ...,
         -3.93747930e-05,  -3.93335013e-06,  -3.93335013e-06],
       [  5.16974264e-06,   1.89559284e-04,   2.32876139e-04, ...,
          1.96624424e-05,  -8.75793778e-06,  -8.75793778e-06],
       ..., 
       [ -2.40926439e-05,  -2.24998449e-06,  -1.24471697e-01, ...,
          9.84018574e-01,   3.57333808e-04,   3.57333808e-04],
       [ -3.43457476e-05,  -5.85791091e-06,  -5.81833020e-06, ...,
          3.59586280e-04,   9.77422214e-01,  -2.25777855e-02],
       [ -3.43457476e-05,  -5.85791091e-06,  -5.81833020e-06, ...,
          3.59586280e-04,  -2.25777855e-02,   9.77422214e-01]])

In [246]:
s

array([ 33.94526536,  25.85151705,  23.87110124,  22.64027177,
        20.35491465,  19.29945065,  18.16558279,  17.82253187,
        17.67981814,  17.01305905,  16.90271382,  16.72914935,
        16.5473897 ,  16.45600448,  16.1895528 ,  15.93422722,
        15.84742248,  15.81407907,  15.76023408,  15.59532241,
        15.54778364,  15.46785625,  15.41594646,  15.29153174,
        15.24790165,  15.10144205,  15.03977686,  14.92602804,
        14.86725361,  14.77708149,  14.72998815,  14.69873388,
        14.67645832,  14.57722379,  14.55966854,  14.47502983,
        14.4487161 ,  14.40360101,  14.35232385,  14.32779146,
        14.21109594,  14.1670542 ,  14.10298862,  14.08773303,
        14.07531339,  14.00155732,  13.95732727,  13.92862514,
        13.87768674,  13.82133281,  13.79359302,  13.76109588,
        13.64501627,  13.62564884,  13.6244154 ,  13.5680165 ,
        13.55123706,  13.48090611,  13.46610527,  13.45762384,
        13.40101395,  13.37671907,  13.34147731,  13.31

In [247]:
np.diag(s)

array([[ 33.94526536,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  25.85151705,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  23.87110124, ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [  0.        ,   0.        ,   0.        , ...,   1.95131862,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          1.86004925,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   1.83050285]])

In [255]:
from numpy import *    

shape(U)

(1000, 1000)

In [264]:
S = np.zeros(original_matrix.shape)

for i in range(len(s)):
    S[i][i] = s[i]

In [265]:
shape(S)

(1000, 30049)

In [266]:
shape(Vt)

(30049, 30049)

U = (1000,1000), S = (1000,30049), Vt = (30049, 30049)

In [267]:
S

array([[ 33.94526536,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,  25.85151705,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  23.87110124, ...,   0.        ,
          0.        ,   0.        ],
       ..., 
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [269]:
U.dot(S).dot(Vt)  #예측값

array([[ -4.07660017e-17,  -1.84937786e-16,   4.71627945e-17, ...,
          2.28224557e-16,   1.09152054e-16,   1.09152054e-16],
       [  7.71951947e-17,   3.87060176e-17,  -2.53703308e-17, ...,
         -2.26847621e-15,   7.04731412e-18,   7.04731412e-18],
       [  3.46944695e-17,  -3.27835632e-17,  -4.60785923e-19, ...,
          1.00000000e+00,  -1.61166653e-16,  -1.61166653e-16],
       ..., 
       [  1.20346441e-17,   3.37034410e-17,  -8.25477018e-17, ...,
         -1.14071621e-16,  -1.68972909e-16,  -1.68972909e-16],
       [ -2.01878445e-16,   1.09070739e-16,  -6.04442711e-17, ...,
         -1.43982049e-16,  -2.89481980e-17,  -2.89481980e-17],
       [  1.11455983e-16,   2.87584626e-17,  -5.00223777e-17, ...,
         -1.07742591e-16,   7.03647210e-17,   7.03647210e-17]])

# 7. TruncatedSVD

In [889]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
svd = TruncatedSVD(n_components=29, n_iter=7)  
U = svd.fit_transform(original_matrix)
Sigma = svd.explained_variance_ratio_
VT = svd.components_
svd.fit(original_matrix)

TruncatedSVD(algorithm='randomized', n_components=29, n_iter=7,
       random_state=None, tol=0.0)

In [890]:
U

array([[  7.83276456e-01,   7.31346079e-01,  -8.14794267e-01, ...,
         -3.50982620e-01,   2.16121548e-02,   9.15120502e-03],
       [  1.55341423e-01,   4.53299458e-01,  -1.61958847e-01, ...,
          2.38771397e-02,  -6.47848760e-03,  -3.99812354e-04],
       [  1.80996367e-02,   2.63191750e-02,  -1.12125603e-02, ...,
         -1.77509029e-02,  -1.10216090e-02,   8.73160772e-03],
       ..., 
       [  1.32543253e+00,   2.51726234e-01,  -6.81430962e-01, ...,
          2.25434712e-01,  -3.75136150e-01,   5.40965028e-02],
       [  4.77815855e-01,   2.10419307e+00,   2.58424850e+00, ...,
         -4.73011876e-01,   3.85680299e-01,   7.35656921e-01],
       [  1.21807109e-01,   7.82824792e-02,  -4.62734927e-02, ...,
         -2.00343659e-02,   1.11067788e-01,   1.45878708e-02]])

In [891]:
Sigma

array([ 0.01239587,  0.00921912,  0.00864232,  0.00761913,  0.00612982,
        0.00554992,  0.0050033 ,  0.00478026,  0.00471241,  0.00438285,
        0.00432082,  0.00421281,  0.00413125,  0.00409735,  0.00395138,
        0.00379972,  0.00375553,  0.00373432,  0.00369063,  0.00365166,
        0.00361892,  0.00357041,  0.00354251,  0.00345133,  0.00341913,
        0.00335467,  0.00332069,  0.00326949,  0.00323655])

In [892]:
Sigma = np.diag(Sigma)

In [893]:
VT

array([[  1.41801912e-06,   2.02974139e-04,   1.39784671e-04, ...,
          1.57081712e-05,   3.06598382e-05,   3.06598382e-05],
       [  8.85867670e-06,   3.23061004e-04,   1.97372098e-04, ...,
          3.93968456e-05,   3.98985172e-06,   3.98985172e-06],
       [ -5.21966801e-06,  -1.90860486e-04,  -2.28295467e-04, ...,
         -1.97146675e-05,   8.77897971e-06,   8.77897971e-06],
       ..., 
       [  5.17303906e-05,  -1.16045416e-03,  -6.77414085e-04, ...,
         -9.51251293e-05,   1.21952210e-04,   1.21952210e-04],
       [  5.97798474e-05,   2.73590166e-04,  -8.94261299e-04, ...,
         -4.14347272e-05,  -4.38389210e-05,  -4.38389210e-05],
       [  7.70265181e-05,  -7.78737862e-04,   2.90271646e-04, ...,
          4.46758472e-05,   9.04498145e-05,   9.04498145e-05]])

In [894]:
shape(U)

(1000, 29)

In [895]:
shape(Sigma)

(29, 29)

In [896]:
shape(VT)

(29, 30049)

In [897]:
U.dot(Sigma).dot(VT)

array([[ -2.51654176e-07,   1.17097017e-05,  -2.23819620e-06, ...,
          5.13768748e-07,   7.04129153e-07,   7.04129153e-07],
       [  3.77253916e-07,   1.50318395e-06,   9.38762109e-06, ...,
          1.65794925e-07,   5.86458224e-07,   5.86458224e-07],
       [ -1.36061237e-08,   4.89895896e-07,  -7.19517808e-08, ...,
          1.97140445e-07,   3.92714160e-08,   3.92714160e-08],
       ..., 
       [ -5.29958617e-08,   1.60294418e-06,  -7.14314433e-06, ...,
         -2.20538517e-08,  -1.22009958e-06,  -1.22009958e-06],
       [  8.07291679e-08,  -4.60231724e-06,  -1.43618518e-05, ...,
          4.20659545e-07,  -1.13183962e-07,  -1.13183962e-07],
       [  4.67681174e-09,   4.73775257e-06,  -5.47210053e-06, ...,
          2.27884970e-07,   7.22543353e-08,   7.22543353e-08]])

# 8. 0-999 json 파일로 만든 training, test data 가져오기

training_data 가져오기

In [638]:
def get_train_data():
    json_data = open('./gwang/train.json').read()
    data = json.loads(json_data)
    df_training_data = pd.DataFrame(data)
    return df_training_data

In [639]:
df_training_data = get_train_data() #training data
df_training_data.head()

,collaborative,description,duration_ms,modified_at,name,num_albums,num_artists,num_edits,num_followers,num_tracks,pid,tracks
0,false,NaN,11532414,1493424000,Throwbacks,47,37,6,1,52,0,"[{u'pos': 0, u'artist_name': u'Missy Elliott',..."
1,false,NaN,11656470,1506556800,Awesome Playlist,23,21,5,1,39,1,"[{u'pos': 0, u'artist_name': u'Survivor', u'tr..."
2,false,NaN,14039958,1505692800,korean,51,31,18,1,64,2,"[{u'pos': 0, u'artist_name': u'Hoody', u'track..."
3,false,NaN,28926058,1501027200,mat,107,86,4,1,126,3,"[{u'pos': 0, u'artist_name': u'Camille Saint-S..."
4,false,NaN,4335282,1401667200,90s,16,16,7,2,17,4,"[{u'pos': 0, u'artist_name': u'The Smashing Pu..."


In [640]:
print(get_train_data().index)
print(get_train_data().columns)

RangeIndex(start=0, stop=900, step=1)
Index([u'collaborative', u'description', u'duration_ms', u'modified_at',
       u'name', u'num_albums', u'num_artists', u'num_edits', u'num_followers',
       u'num_tracks', u'pid', u'tracks'],
      dtype='object')


training data set(900개) track 합치기

In [641]:
def concat_training_data_tracks(dataframe):
    df_tracks = pd.DataFrame(dataframe['tracks'][0])
    for i in range(1,900):
        df_tmp = DataFrame(dataframe['tracks'][i])
        df_tracks = pd.concat([df_tracks, df_tmp])
    df_tracks = df_tracks.reset_index(drop=True)
    return df_tracks

In [642]:
df_training_tracks = concat_training_data_tracks(df_training_data)

In [643]:
df_training_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM


test_data 가져오기

In [644]:
def get_test_data():
    json_data = open('./gwang/test.json').read()
    data = json.loads(json_data)
    df_test_data = pd.DataFrame(data)
    return df_test_data

In [645]:
df_test_data = get_test_data() #test data

In [840]:
df_test_data

,name,num_holdouts,num_samples,num_tracks,pid,tracks
0,2000s hits,28,25,53,663,"[{u'pos': 0, u'artist_name': u'Fergie', u'trac..."
1,LM,2,25,27,385,"[{u'pos': 0, u'artist_name': u'Jason Mraz', u'..."
2,work,25,25,50,860,"[{u'pos': 0, u'artist_name': u'Jens Lekman', u..."
3,nite,35,25,60,583,"[{u'pos': 0, u'artist_name': u'PARTYNEXTDOOR',..."
4,Go To,63,25,88,392,"[{u'pos': 0, u'artist_name': u'Jessie J', u'tr..."
5,w o r k o u t,220,25,245,123,"[{u'pos': 0, u'artist_name': u'Trap Beckham', ..."
6,deep,80,25,105,302,"[{u'pos': 0, u'artist_name': u'Band of Horses'..."
7,workout,76,25,101,621,"[{u'pos': 0, u'artist_name': u'Zookeepers', u'..."
8,Piano Music,26,25,51,407,"[{u'pos': 0, u'artist_name': u'Jim Brickman', ..."
9,cake,22,25,47,670,"[{u'pos': 0, u'artist_name': u'Niykee Heaton',..."


# 9. new df_track with playlist idx 

test_data에 있는 pid playlist idx에서 지우기 ( * list에 track수가 25개 넘지 않는 리스트는 제외했으므로 test data는 100개의 list가 아님)

In [647]:
num_tracks = df_training_data["num_tracks"]
pid_li = df_training_data['pid']
playlist_pid = []
for i in range(len(pid_li)):
    tmp = [pid_li[i] for _ in range(num_tracks[i])]
    playlist_pid = playlist_pid + tmp
df_training_tracks['playlist_pid'] = playlist_pid

df_training_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_pid
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0


# 10. unique한 track 리스트에 담기

test의 트랙도 포함시킬것

In [648]:
tracks_of_train_set = df_training_tracks['track_name']

test set 트랙 합치기

In [660]:
df_tracks = pd.DataFrame(df_test_data['tracks'][0])

for i in range(1, size(df_test_data['tracks'])):
    df_tmp = DataFrame(df_test_data['tracks'][i])
    df_tracks = pd.concat([df_tracks , df_tmp])
df_test_tracks = df_tracks.reset_index(drop=True)

In [661]:
df_test_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri
0,The Dutchess,spotify:album:0jwuTvP3hp2jFY08VLgvnD,Fergie,spotify:artist:3r17AfJCCUqC9Lf0OAc73G,268120,0,Big Girls Don't Cry (Personal),spotify:track:3Q4WeJmzxuDpzMu9QjQqbM
1,The Blueprint 3,spotify:album:22v70dJq6kWS32SkR2c7kd,JAY Z,spotify:artist:3nFkdlSjzX9mRTtwJOzDYB,276920,1,Empire State Of Mind,spotify:track:3lPAHV0b53zgRRm9LBgnTY
2,Elephunk,spotify:album:1bNyYpkDRovmErm4QeDrpJ,The Black Eyed Peas,spotify:artist:1yxSLGMDHlW21z4YXirZDS,272533,2,Where Is The Love?,spotify:track:0uqPG793dkDDN7sCUJJIVC
3,Shine,spotify:album:4nAIqmKELnBEXEkYg1pMic,Estelle,spotify:artist:5T0MSzX9RC5NA6gAI6irSn,284733,3,American Boy (feat. Kanye West),spotify:track:41on8RwRh22IHcChAN2gm8
4,Tasty,spotify:album:7zesXMFikT4DdgkklIk3Jz,Kelis,spotify:artist:0IF46mUS8NXjgHabxk2MCM,182626,4,Milkshake,spotify:track:2cMTIlktg3M9mXYqCPqw1J
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4
6,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,275986,6,Umbrella,spotify:track:5i66xrvSh1MjjyDd6zcwgj
7,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,267080,7,Don't Stop The Music,spotify:track:62LHRv9uwSNlBmByQF5jdE
8,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,195040,8,Push Up On Me,spotify:track:56y79SaKYtK2mTC9PqYzkR
9,Raymond v Raymond (Deluxe Edition),spotify:album:6A1F3Fkq5dYeYYNkXflcTX,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,220800,9,DJ Got Us Fallin' In Love,spotify:track:4356Typ82hUiFAynbLYbPn


In [662]:
tracks_of_test_set = df_test_tracks['track_name']

In [663]:
unique_tracks = pd.concat([tracks_of_train_set, tracks_of_test_set]) # train과 test set track 합치기
unique_tracks = pd.DataFrame(unique_tracks)

In [664]:
unique_tracks = unique_tracks.drop_duplicates(['track_name'],keep='first') # 중복제거

In [876]:
unique_tracks.head(10)

,track_name
0,Lose Control (feat. Ciara & Fat Man Scoop)
1,Toxic
2,Crazy In Love
3,Rock Your Body
4,It Wasn't Me
5,Yeah!
6,My Boo
7,Buttons
8,Say My Name
9,Hey Ya! - Radio Mix / Club Mix


In [665]:
len(unique_tracks)

28809

# 11. matrix만들기

training data로 matrix만들기

In [724]:
df_training_tracks['exist'] = 100

In [725]:
df_training_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_pid,exist
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0,100
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0,100
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0,100
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0,100
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0,100
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0,100
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0,100
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0,100
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0,100
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0,100


test data로 matrix만들기

In [726]:
df_test_tracks['exist'] = 100

playlist_pid column 추가

In [727]:
num_tracks = df_test_data["num_tracks"]
pid_li = df_test_data['pid']
playlist_pid = []
for i in range(len(pid_li)):
    tmp = [pid_li[i] for _ in range(25)]
    playlist_pid = playlist_pid + tmp

df_test_tracks['playlist_pid'] = playlist_pid
df_test_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,exist,playlist_pid
0,The Dutchess,spotify:album:0jwuTvP3hp2jFY08VLgvnD,Fergie,spotify:artist:3r17AfJCCUqC9Lf0OAc73G,268120,0,Big Girls Don't Cry (Personal),spotify:track:3Q4WeJmzxuDpzMu9QjQqbM,100,663
1,The Blueprint 3,spotify:album:22v70dJq6kWS32SkR2c7kd,JAY Z,spotify:artist:3nFkdlSjzX9mRTtwJOzDYB,276920,1,Empire State Of Mind,spotify:track:3lPAHV0b53zgRRm9LBgnTY,100,663
2,Elephunk,spotify:album:1bNyYpkDRovmErm4QeDrpJ,The Black Eyed Peas,spotify:artist:1yxSLGMDHlW21z4YXirZDS,272533,2,Where Is The Love?,spotify:track:0uqPG793dkDDN7sCUJJIVC,100,663
3,Shine,spotify:album:4nAIqmKELnBEXEkYg1pMic,Estelle,spotify:artist:5T0MSzX9RC5NA6gAI6irSn,284733,3,American Boy (feat. Kanye West),spotify:track:41on8RwRh22IHcChAN2gm8,100,663
4,Tasty,spotify:album:7zesXMFikT4DdgkklIk3Jz,Kelis,spotify:artist:0IF46mUS8NXjgHabxk2MCM,182626,4,Milkshake,spotify:track:2cMTIlktg3M9mXYqCPqw1J,100,663
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,100,663
6,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,275986,6,Umbrella,spotify:track:5i66xrvSh1MjjyDd6zcwgj,100,663
7,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,267080,7,Don't Stop The Music,spotify:track:62LHRv9uwSNlBmByQF5jdE,100,663
8,Good Girl Gone Bad,spotify:album:1YhbfKnjrFgnYyWz6cn9mN,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,195040,8,Push Up On Me,spotify:track:56y79SaKYtK2mTC9PqYzkR,100,663
9,Raymond v Raymond (Deluxe Edition),spotify:album:6A1F3Fkq5dYeYYNkXflcTX,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,220800,9,DJ Got Us Fallin' In Love,spotify:track:4356Typ82hUiFAynbLYbPn,100,663


In [728]:
total_tracks = pd.concat([df_training_tracks, df_test_tracks])  # training과 test 합치기

In [729]:
total_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,exist,playlist_pid,pos,track_name,track_uri
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,100,0,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,100,0,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,100,0,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,100,0,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,100,0,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,100,0,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,100,0,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,100,0,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,100,0,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,100,0,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM


#  여기 0번부터 999번째 파일 (총 1000개) tracks concat

In [967]:
def concat_file(last_file_num):
    df_file_tracks = DataFrame(tracks_with_playlist_idx(0))
    for i in range(1, last_file_num):
        df_file_tmp = pd.DataFrame(tracks_with_playlist_idx(i))
        df_file_tracks = pd.concat([df_file_tracks, df_file_tmp])
    return df_file_tracks.reset_index(drop=True)

In [974]:
concat_file(100)

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_idx
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0


In [975]:
concat_100_tracks = concat_file(100)

In [976]:
concat_100_tracks

,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,track_uri,playlist_idx
0,The Cookbook,spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,Missy Elliott,spotify:artist:2wIVse2owClT7go1WT98tk,226863,0,Lose Control (feat. Ciara & Fat Man Scoop),spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,0
1,In The Zone,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,198800,1,Toxic,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,0
2,Dangerously In Love (Alben für die Ewigkeit),spotify:album:25hVFAxTlDvXbx2X2QkUkE,Beyoncé,spotify:artist:6vWDO969PvNqNYHIOW5v0m,235933,2,Crazy In Love,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,0
3,Justified,spotify:album:6QPkyl04rXwTGlGlcYaRoW,Justin Timberlake,spotify:artist:31TPClRtHm23RisEBtV3X7,267266,3,Rock Your Body,spotify:track:1AWQoqb9bSvzTjaLralEkT,0
4,Hot Shot,spotify:album:6NmFmPX56pcLBOFMhIiKvF,Shaggy,spotify:artist:5EvFsr3kj42KNv97ZEnqij,227600,4,It Wasn't Me,spotify:track:1lzr43nnXAijIGYnCT8M8H,0
5,Confessions,spotify:album:0vO0b1AvY49CPQyVisJLj0,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,250373,5,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,0
6,Confessions,spotify:album:1RM6MGv6bcl6NrAG8PGoZk,Usher,spotify:artist:23zg3TcAtWQy7J6upgbUnj,223440,6,My Boo,spotify:track:68vgtRHr7iZHpzGpon6Jlo,0
7,PCD,spotify:album:5x8e8UcCeOgrOzSnDGuPye,The Pussycat Dolls,spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ,225560,7,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,0
8,The Writing's On The Wall,spotify:album:283NWqNsCA9GwVHrJk59CG,Destiny's Child,spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i,271333,8,Say My Name,spotify:track:7H6ev70Weq6DdpZyyTmUXk,0
9,Speakerboxxx/The Love Below,spotify:album:1UsmQ3bpJTyK6ygoOOjG1r,OutKast,spotify:artist:1G9G7WwrXka3Z1r7aIDjI7,235213,9,Hey Ya! - Radio Mix / Club Mix,spotify:track:2PpruBYCo4H7WOBJ7Q2EwM,0


In [978]:
store = concat_100_tracks.T.to_dict().values()


In [982]:
concat_100_tracks.to_json('./concat_100_tracks.json')

In [984]:
concat_100_tracks.shape

(6677800, 9)

In [ ]:
concat_200_tracks = concat_file(200)

In [980]:
with open('./concat_tracks_of_100_file.json', 'w+') as f:
    f.write(json.dumps(store))

In [ ]:
concat_100_tracks = concat_file(200)

In [ ]:
common_front = '../data/mpd.slice.'
common_back = '.json'


def get_playlist_file(file_num):   
    front = file_num*1000
    back = front + 999
    file_path = common_front + str(front) + '-' +str(back) + common_back
    json_data = open(file_path).read()
    data = json.loads(json_data)
    playlists = data["playlists"]
    df_playlist = pd.DataFrame(playlists)
    return df_playlist

In [730]:
matrix = total_tracks.pivot_table(index='playlist_pid', columns='track_name', values = 'exist').fillna(0)

In [731]:
matrix

track_name,"""40"" - Remastered 2008","""45""","""A Lovely Night""","""A Lovely Night"" (Reprise)","""Can't Repeat The Past?""","""Do I Love You Because You're Beautiful""","""Esh 'Dani, Alash Mshit""","""Gatsby Believed In The Green Light""","""He Was Tall""","""Heroes"" - Single Version, 2017 Remastered Version",...,사실은 The Truth Is,양화대교 (Yanghwa Brdg),오늘 헤어졌어요,왜 (Keep Your Head Down) [Rearranged],왜 이러니,우주보다 좋아해 (I Love You More Than Universe),으르렁 Growl,전야 前夜 The Eve,주문 Mirotic,"헤어지지 못하는 여자, 떠나가지 못하는 남자 Can't Breakup Girl, Can't Breakaway Boy (feat. 정인 Jung In)"
playlist_pid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,100.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 12. Truncated SVD

In [905]:
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
svd = TruncatedSVD(n_components=45, n_iter=7)  
U = svd.fit_transform(matrix)
Sigma = svd.explained_variance_ratio_
VT = svd.components_
svd.fit(matrix)

TruncatedSVD(algorithm='randomized', n_components=45, n_iter=7,
       random_state=None, tol=0.0)

In [906]:
U

array([[  7.41837410e+01,   7.89036726e+01,  -6.95267574e+01, ...,
         -2.38502927e+01,   1.27528992e+00,   4.26376281e+00],
       [  1.38158662e+01,   6.11218163e+01,   3.76151694e+01, ...,
          1.64473034e+00,   1.58461809e+00,  -4.65507248e-01],
       [  1.87480152e+00,   3.13624239e+00,  -4.22836113e-01, ...,
         -1.76568384e-01,  -5.51685535e+00,   3.35605185e-01],
       ..., 
       [  1.27789873e+02,   2.89392632e+01,  -7.66363944e+01, ...,
          1.85263912e+01,   2.08729764e-01,   1.98202507e+01],
       [  4.67262855e+01,   1.65574269e+02,   2.50265981e+02, ...,
          4.96908943e+01,   1.35126291e+01,   3.55278197e+01],
       [  1.29863256e+01,   9.21660863e+00,  -3.76255572e+00, ...,
         -3.48301689e+00,  -8.58669312e+00,  -4.52480709e+00]])

In [907]:
Sigma

array([ 0.012326  ,  0.00894637,  0.00802279,  0.00790602,  0.00623918,
        0.00575593,  0.00519092,  0.00504268,  0.00486869,  0.00454415,
        0.00450256,  0.0044254 ,  0.00427267,  0.00425669,  0.00413153,
        0.00396776,  0.00391396,  0.00387179,  0.00390033,  0.00387761,
        0.00380416,  0.00373225,  0.00370377,  0.00365   ,  0.00356752,
        0.00355671,  0.00351461,  0.00347366,  0.00346201,  0.0034461 ,
        0.0033846 ,  0.00334993,  0.00332649,  0.00329958,  0.00326906,
        0.00325403,  0.00321192,  0.00317   ,  0.00314614,  0.00311733,
        0.00310741,  0.0030797 ,  0.0030461 ,  0.0030305 ,  0.00300596])

In [908]:
Sigma = np.diag(Sigma)

In [909]:
VT

array([[  1.54978175e-06,   2.20346543e-04,   1.51488245e-04, ...,
          1.75941189e-05,   3.60841905e-05,   3.60841905e-05],
       [  1.31610965e-05,   4.34201083e-04,   2.33922689e-04, ...,
          5.09436205e-05,   9.13463648e-06,   9.13463648e-06],
       [  9.44455830e-06,  -8.61972186e-05,  -2.60047546e-04, ...,
         -8.40296546e-06,   3.77994803e-05,   3.77994803e-05],
       ..., 
       [  1.75686690e-04,  -8.23939711e-05,   1.20146858e-04, ...,
         -6.49674166e-06,  -1.33826856e-04,  -1.33826856e-04],
       [ -1.11318639e-05,   7.91804491e-04,   4.29523881e-04, ...,
         -2.97401173e-04,  -1.70804754e-04,  -1.70804754e-04],
       [ -1.97707978e-04,   2.44288977e-03,  -9.93898342e-06, ...,
          2.41998849e-05,  -1.54044012e-04,  -1.54044012e-04]])

In [910]:
shape(U)

(979, 45)

In [911]:
shape(Sigma)

(45, 45)

In [912]:
shape(VT)

(45, 28809)

In [913]:
result = U.dot(Sigma).dot(VT)
result

array([[ -1.53091330e-05,   4.30394946e-04,   3.37081043e-04, ...,
         -1.84030874e-05,   1.13778743e-04,   1.13778743e-04],
       [  4.13492929e-05,   2.44097475e-04,   7.78623903e-04, ...,
          7.91596423e-06,   7.06358912e-05,   7.06358912e-05],
       [  3.53862064e-06,   7.55708931e-05,  -1.49447710e-04, ...,
          5.30778592e-05,   5.89952607e-06,   5.89952607e-06],
       ..., 
       [  2.56092493e-05,   1.57619190e-03,  -4.01552590e-04, ...,
          1.64673240e-05,  -1.79830333e-04,  -1.79830333e-04],
       [ -4.36474264e-05,  -2.72931123e-04,  -2.03155830e-04, ...,
         -1.94019396e-05,  -1.17800873e-05,  -1.17800873e-05],
       [ -5.96179688e-06,   2.78551972e-04,  -1.85652607e-04, ...,
          4.98680317e-05,   8.39565961e-06,   8.39565961e-06]])

In [914]:
len(result) #row개수

979

In [915]:
len(result[0]) # column 개수

28809

In [916]:
result[978].max()  #최대 rank값

0.04330770765530477

In [917]:
output = result[900:979]
output

array([[ -1.76033901e-05,  -6.30041815e-04,   6.24302653e-04, ...,
         -4.11504755e-05,   1.79377644e-05,   1.79377644e-05],
       [ -6.15000570e-06,   1.96058986e-03,  -5.04287147e-03, ...,
          1.75657485e-04,  -2.12692756e-04,  -2.12692756e-04],
       [  2.36099596e-05,   6.20439761e-04,  -1.49012139e-04, ...,
          7.09500337e-05,   4.25190592e-05,   4.25190592e-05],
       ..., 
       [  2.56092493e-05,   1.57619190e-03,  -4.01552590e-04, ...,
          1.64673240e-05,  -1.79830333e-04,  -1.79830333e-04],
       [ -4.36474264e-05,  -2.72931123e-04,  -2.03155830e-04, ...,
         -1.94019396e-05,  -1.17800873e-05,  -1.17800873e-05],
       [ -5.96179688e-06,   2.78551972e-04,  -1.85652607e-04, ...,
          4.98680317e-05,   8.39565961e-06,   8.39565961e-06]])

# Matrix Factorization 으로 예측한 각 플레이리스트 마다의 50개의 추천트랙생성



# Test set에서 주어진 25개 트랙은 빼고 추천하기 위해서 25개 트랙의 행렬 값을 최소값으로 수정

In [928]:
import numpy as np

mx_col = matrix.columns.tolist()
track_idxs = []

# print df_test_data
for i in range(df_test_data.shape[0]):
    track_idx = []
    for j in range(25):
        track_idx.append(mx_col.index((df_test_data['tracks'].loc[[i]]).tolist()[0][j]['track_name']))
    track_idxs.append(track_name)
    
    
for i in range(len(track_idxs)):
    for idx in track_idxs[i]:
        output[i][idx] = -10

              name  num_holdouts  num_samples  num_tracks  pid  \
0       2000s hits            28           25          53  663   
1               LM             2           25          27  385   
2             work            25           25          50  860   
3             nite            35           25          60  583   
4            Go To            63           25          88  392   
5    w o r k o u t           220           25         245  123   
6             deep            80           25         105  302   
7          workout            76           25         101  621   
8      Piano Music            26           25          51  407   
9            cake             22           25          47  670   
10              <3            10           25          35  939   
11           Music            60           25          85  183   
12            maY!            34           25          59  596   
13       classical            35           25          60  667   
14  The Pi

# Matrix Factorization 으로 구한 행렬을 이용해 가장 높은 값을 가진 50개의 트랙 추출

In [919]:
rec_track_names = []

for i in range(len(output)):
    ranking = np.argsort(output[i])
    rec_track_name = []
    for dat in reversed(ranking):
        if len(rec_track_name) < 50:
            rec_track_name.append(mx_col[dat])
    rec_track_names.append(rec_track_name)
  


# track_name - > track_uri

In [920]:
rec_track_uris = []

for i in range(len(rec_track_names)):
    rec_track_uri = []
    for dat in rec_track_names[i]:
        rec_track_uri.append(total_tracks.loc[total_tracks['track_name'] == dat].iloc[[0]]['track_uri'].tolist()[0])
    rec_track_uris.append(rec_track_uri)

# Submission file format = ['pid', 'track_uri1','track_uri2' , ,,,,,, ]

In [921]:
pid_li = df_test_data['pid'].tolist()

for i in range(len(rec_track_uris)):
    rec_track_uris[i].insert(0,pid_li[i])
    
rec_track_uris

[[663,
  u'spotify:track:7yyRTcZmCiyzzJlNzGC9Ol',
  u'spotify:track:2nbClS09zsIAqNkshg6jnp',
  u'spotify:track:1xznGGDReH1oQq0xzbwXa3',
  u'spotify:track:0QsvXIfqM0zZoerQfsI9lm',
  u'spotify:track:0v9Wz8o0BT8DU38R4ddjeH',
  u'spotify:track:3a1lNhkSLSkpJE4MSHpDu9',
  u'spotify:track:2EEeOnHehOozLq4aS0n6SL',
  u'spotify:track:11KJSRSgaDxqydKYiD2Jew',
  u'spotify:track:6O6M7pJLABmfBRoGZMu76Y',
  u'spotify:track:6eT7xZZlB2mwyzJ2sUKG6w',
  u'spotify:track:12REd1n8PeiHHWOh066tpr',
  u'spotify:track:6DNtNfH8hXkqOX1sjqmI7p',
  u'spotify:track:6gBFPUFcJLzWGx4lenP6h2',
  u'spotify:track:112lQZi5e7i8ohZ4z0e6nu',
  u'spotify:track:27GmP9AWRs744SzKcpJsTZ',
  u'spotify:track:5hTpBe8h35rJ67eAWHQsJx',
  u'spotify:track:5uDASfU19gDxSjW8cnCaBp',
  u'spotify:track:0SGkqnVQo9KPytSri1H6cF',
  u'spotify:track:6vnDoUOHTiVB5DvX18tJon',
  u'spotify:track:69bp2EbF7Q2rqc5N3ylezZ',
  u'spotify:track:7KXjTSCq5nL1LoYtL7XAwS',
  u'spotify:track:7EiZI6JVHllARrX9PUvAdX',
  u'spotify:track:4Km5HrUvYTaSUfiSGPJeQR',
  u'

# 최종적으로 제출할 submission.csv 파일 생성

In [922]:
import csv

with open("./gwang/output.csv", "w+") as f:
    writer = csv.writer(f)
    writer.writerow(np.arange(len(rec_track_uris[i])))
    writer.writerows(rec_track_uris)
    

# 제출하여 오류 확인 

# 광진오빠 코드  

In [929]:
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [966]:
def make_category_1(file_path):


    f = open(file_path).read()
    data = json.loads(f)
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)

    r_test_data = {'pid' : list(test['pid']), 'name' : list(test['name']), 'num_holdouts' : list(test['num_tracks']), 'tracks' : [{} for _ in range(test.shape[0])], 'num_samples' : [0 for _ in range(test.shape[0])], 'num_tracks' : test['num_tracks']}
    r_test = pd.DataFrame(r_test_data)
    
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
    

def make_category_7(file_path):


    f = open(file_path).read()
    data = json.loads(f) 
    playlists = data["playlists"]
    df = pd.DataFrame(playlists)
    train, test = train_test_split(df, test_size=0.1)
    
    sub_tracks = test['tracks'].tolist()
    subb_tracks = []
    include_li = []  # it has playlists which has more than 
    exclude_li = []
    
    #when the number of tracks is more than 25, we store playlists into test set
    #if we can store it, we store 25 tracks from the beginning track
    for i in range(len(sub_tracks)):
        if len(sub_tracks[i]) > 25:
            subb_tracks.append(sub_tracks[i][:25])
            include_li.append(i)
        else:
            exclude_li.append(i)
    
   
    li_pid =[]
    li_name = []
    li_num_holdouts = []
    li_num_samples = []
    li_num_tracks = []
    
    print type((test['pid']))
    # loop for testset's length 
    # add attribute
    for i in range(test.shape[0]):
        if i in include_li:
            li_pid.append(list(test['pid'])[i])
            li_name.append(list(test['name'])[i])
            li_num_holdouts.append(list(test['num_tracks'])[i]-25)
            li_num_samples.append(25)
            li_num_tracks.append(list(test['num_tracks'])[i])
    
    r_test_data = {'pid' : li_pid, 'name' : li_name, 'num_holdouts' : li_num_holdouts, 'tracks' : subb_tracks, 'num_samples' : li_num_samples, 'num_tracks' : li_num_tracks}
    
    
    r_test = pd.DataFrame(r_test_data)
    test = test.drop(test.index[exclude_li])
    train = train.T.to_dict().values()
    test = test.T.to_dict().values()
    r_test = r_test.T.to_dict().values()
    
    with open('./train.json', 'w+') as f:
        f.write(json.dumps(train))
    with open('./test_R.json', 'w+') as f:
        f.write(json.dumps(test))
    with open('./test.json', 'w+') as f:
        f.write(json.dumps(r_test))
    
  


make_category_7('../data/mpd.slice.0-999.json')


<class 'pandas.core.series.Series'>


TypeError: 'list' object is not callable

# 0~1000 playlist -> 900개 training set, 100개 -> test set 으로 구별

In [925]:
f = open('./train.json').read()
data = json.loads(f)
df = pd.DataFrame(data)
df

IOError: [Errno 2] No such file or directory: './train.json'

In [924]:
f = open('./test.json').read()
data = json.loads(f)
df = pd.DataFrame(data)
df

IOError: [Errno 2] No such file or directory: './test.json'